In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

In [60]:
# Загрузка данных
X_train = np.load('data/X_train.npy')
X_test = np.load('data/X_test.npy')
y_train = pd.read_csv('data/y_train.csv')

# Отображение размерностей данных и первых несколько строк y_train
X_train_shape = X_train.shape
X_test_shape = X_test.shape
y_train_shape = y_train.shape
y_train_head = y_train.head()

print('Размер тренировочной выборки: {}'.format(X_train_shape), 'Размер тестовой выборки: {}'.format(X_test_shape), 'Метки классов, размер: {}'.format(y_train_shape), sep='\n')

display(y_train_head)


Размер тренировочной выборки: (323, 40, 100)
Размер тестовой выборки: (54, 40, 100)
Метки классов, размер: (32300, 2)


,sample-timestep,class
0,0-0,0
1,0-1,0
2,0-2,0
3,0-3,0
4,0-4,0


In [61]:
# Анализ распределения классов в y_train
class_distribution = y_train['class'].value_counts().sort_index()

class_distribution


class
0    17950
1     1800
2     1800
3     1800
4     1750
5     1800
6     1800
7     1800
8     1800
Name: count, dtype: int64

In [62]:
# Сделаем датафрейм из тренировочных данных. Датафрейм будет состоя из 40 столбцов и 32300 строчек, и будет содеражть данные 
# со всем датчиков по сто значений на каждый семпл
X_train_df = pd.DataFrame(columns=[f'{i}'for i in range(X_train.shape[1])])
for i in range(len(X_train)):
    X_train_df = pd.concat([X_train_df, pd.DataFrame(X_train[i].T, columns=[f'{i}'for i in range(X_train.shape[1])])], ignore_index=True)

X_test_df = pd.DataFrame(columns=[f'{i}'for i in range(X_test.shape[1])])
for i in range(len(X_test)):
    X_test_df = pd.concat([X_test_df, pd.DataFrame(X_test[i].T, columns=[f'{i}'for i in range(X_test.shape[1])])], ignore_index=True)
    
# Выведем размеры полученных датафреймов
print('Размер тренировочной выборки:', X_train_df.shape)
print('Размер тестовой выборки:', X_test_df.shape)

# Запишем полученные датафреймы в файлы
#submission_file_path = 'data/X_train_df.csv'
#X_train_df.to_csv(submission_file_path, index=False)

#submission_file_path = 'data/X_test_df.csv'
#X_test_df.to_csv(submission_file_path, index=False)

Размер тренировочной выборки: (32300, 40)
Размер тестовой выборки: (5400, 40)


In [63]:
# Стандартизация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)

y = y_train['class']

# Разделение на обучающую и валидационную выборки
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_train_split.shape, X_val_split.shape, y_train_split.shape, y_val_split.shape


((25840, 40), (6460, 40), (25840,), (6460,))

In [ ]:
# Нормализация данных
scaler = MinMaxScaler()

# Нормализация обучающих и тестовых данных
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [65]:
# Попробуем решить задачу с использованием XGBboosting и используем Optuna для поиска оптимальных параметров

# Определение пространства гиперпараметров для поиска
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5, 0.7],
    'n_estimators': [50, 100, 150, 1000],
    'max_depth': [3, 5, 7, 10]
}

# Модель XGBoost для GridSearch
xgb_model_gs = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Настройка GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model_gs, param_grid=param_grid, cv=5, scoring='f1_weighted', verbose=2, n_jobs=-1)

# Запуск поиска
grid_search.fit(X_train_split, y_train_split)

# Лучшие параметры и лучшая оценка
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print('Лучшие параметры:', best_params)
print('Лучший скор:', best_score)



Fitting 5 folds for each of 64 candidates, totalling 320 fits


({'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 1000},
 0.9663945708836053)

In [66]:
# Проверим на отложенной выборке, как работает модель

xgb_model_gs = XGBClassifier(*best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model_gs.fit(X_train_split, y_train_split)
predictions = xgb_model_gs.predict(X_val_split)
f1_val = f1_score(y_val_split, predictions, average='weighted')
print('Значение F1 метрики для валидационного набора данных:', f1_val)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


0.9653112343120472

In [70]:
xgb_submission = pd.read_csv('data/sample_submission.csv')
predictions = grid_search.predict(X_test_df)
xgb_submission['class'] = predictions
# Сохраняем DataFrame в CSV файл без индекса
submission_file_path = 'data/submission_xgboost_v3.csv'
xgb_submission.to_csv(submission_file_path, index=False)

In [69]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Функция для определения объективной метрики, которую Optuna будет оптимизировать
def objective(trial):
    # Определение диапазона значений гиперпараметров
    param = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.4),
        'depth': trial.suggest_int('depth', 4, 11),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 11),
        'loss_function': 'MultiClass',
        'eval_metric': 'TotalF1:average=Weighted',
        'verbose': False
    }
    
       
    # Создание и обучение модели CatBoost
    model = CatBoostClassifier(**param)
    model.fit(X_train_split, y_train_split, eval_set=[(X_val_split, y_val_split)], early_stopping_rounds=10, verbose=0)
    
    # Предсказание на валидационном наборе
    preds = model.predict(X_val_split)
    
    # Вычисление F1-скора
    f1 = f1_score(y_val_split, preds, average='weighted')
    
    return f1

# Создание Optuna study объекта и запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # Укажите количество итераций поиска

# Вывод лучших параметров
print("Лучшие параметры:", study.best_params)
print("Лучший F1-скор:", study.best_value)

[I 2024-04-10 20:28:55,615] A new study created in memory with name: no-name-270bc55c-3d6e-4708-9f5d-42fa4f48c5dc
[I 2024-04-10 20:29:01,117] Trial 0 finished with value: 0.8834937157326059 and parameters: {'iterations': 78, 'learning_rate': 0.3578071298379829, 'depth': 8, 'l2_leaf_reg': 11}. Best is trial 0 with value: 0.8834937157326059.
[I 2024-04-10 20:29:30,370] Trial 1 finished with value: 0.9087474093245724 and parameters: {'iterations': 87, 'learning_rate': 0.215400184889273, 'depth': 10, 'l2_leaf_reg': 4}. Best is trial 1 with value: 0.9087474093245724.
[I 2024-04-10 20:30:28,113] Trial 2 finished with value: 0.9247981871924212 and parameters: {'iterations': 172, 'learning_rate': 0.18284857319707826, 'depth': 10, 'l2_leaf_reg': 7}. Best is trial 2 with value: 0.9247981871924212.
[I 2024-04-10 20:30:34,194] Trial 3 finished with value: 0.853486855812969 and parameters: {'iterations': 81, 'learning_rate': 0.17059745653062813, 'depth': 8, 'l2_leaf_reg': 11}. Best is trial 2 with 

Лучшие параметры: {'iterations': 200, 'learning_rate': 0.28492494272990826, 'depth': 11, 'l2_leaf_reg': 2}
Лучший F1-скор: 0.9433101917701743


In [71]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score

# Создание объекта модели CatBoostClassifier
catboost_model_best = CatBoostClassifier(
    iterations=225,
    learning_rate=0.28868655799528853,
    depth= 11,
    l2_leaf_reg=7,
    loss_function='MultiClass',
    verbose=False,
    eval_metric='TotalF1:average=Weighted'
)

# Обучение модели
catboost_model_best.fit(X_train_split, y_train_split, eval_set=[(X_val_split, y_val_split)], early_stopping_rounds=10, verbose=0)

# Предсказание на валидационном наборе
preds = catboost_model_best.predict(X_val_split)
    
    # Вычисление F1-скора
f1 = f1_score(y_val_split, preds, average='weighted')

print(f"F1-Score on Validation Set: {f1}")

F1-Score on Validation Set: 0.9474960167651263


In [ ]:
print(f"F1-Score on Validation Set: {f1}")

F1-Score on Validation Set: 0.9474960167651263


In [74]:
preds = catboost_model_best.predict(X_test_df)
submission_catboost_v3 = pd.read_csv('data/sample_submission.csv')
submission_catboost_v3['class'] = preds
# Сохраняем DataFrame в CSV файл без индекса
submission_file_path = 'data/submission_catboost_v3.csv'
submission_catboost_v3.to_csv(submission_file_path, index=False)